<a href="https://colab.research.google.com/github/hihohub/umbrella-networks/blob/main/softmax.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

- Umbrella networks
- copyright 2021 by James Smith
- contact - derekjsmit@gmail.com

Demonstration instructions
- How to organize a 'softmax' umbrella network with models that predict multiple classes.
- Organize your image files into folders by category. For example, (book covers) -> (scifi), (fantasy), (mystery).
- First, run the large cell containing the umbrella code, then scroll down 1333 lines to the experiment.
- Change the folder name in the demonstration code to your folder path.
- The rest of the code should work as written.
- Set 'levels' to anywhere from 1 to the number of layers in your folder structure. If levels=1, images from lower level folders are compressed into the outer level folders. If levels > 1, umbrella networks are formed recursively, one for each outer folder and each subfolder within the outer folders, up to the specified number of levels. The inner folders do not each require the same number of levels, but must have at least as many levels as the value specified for 'levels'.

Umbrella network instructions<br/>
- The umbrella code was designed for reading a local folder and automatically constructing the hierarchical network from the implicit hierarchy in the subfolders.
- If have images in folders, initialize with images_folder = path. Umbrella network constructs tree by itself. Refer to examples.
- However, it also has functions for reading the data from an online repository and building the tree node by node (since the data has no folder structure).
- If loading data from repository, initialize with images_folder = None, then use repo functions. Must first preprocess the data (normalize, reshape). Only process x data, not labels. Umbrella network constructs labels by itself. However, subdivide classes by label, then manually construct tree with repo functions. Finally, init_from_repo(). Refer to examples.
- In repo mode, umbrella network throws away labels and makes its own. For example, if divide MNIST into odd and even numbers, each subset will have labels 0-4. Hence, anything other than flat topology requires that the umbrella network form new labels.






In [1]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


%%%%%%%%%%%%%%%%%%<br/>
UMBRELLA NETWORK<br/>
%%%%%%%%%%%%%%%%%%<br/>

In [2]:
! wget https://raw.githubusercontent.com/hihohub/umbrella-networks/main/umbrella.py

--2022-04-15 04:55:46--  https://raw.githubusercontent.com/hihohub/umbrella-networks/main/umbrella.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.111.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 48219 (47K) [text/plain]
Saving to: ‘umbrella.py’

umbrella.py         100%[===================>]  47.09K  --.-KB/s    in 0.001s  

2022-04-15 04:55:46 (73.9 MB/s) - ‘umbrella.py’ saved [48219/48219]



In [3]:
from umbrella import Umbrella_Network,Umbrella_Label,Umbrella_Node,Joint_Probability_Network

In [4]:
import os
import sys
import numpy as np
import math
import random
from PIL import Image, ImageOps, ImageChops
import cv2
import tensorflow as tf
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Sequential, Model, load_model
from tensorflow.keras.layers import Input, Conv2D, MaxPool2D, Flatten, Dense, add, BatchNormalization, Dropout, AveragePooling2D, GlobalAveragePooling2D, SeparableConv2D
from tensorflow.keras.utils import to_categorical
from tensorflow.python.framework.ops import disable_eager_execution
from tensorflow.keras import backend as backend
from keras.callbacks import ReduceLROnPlateau
from sklearn.utils import shuffle
import matplotlib.pyplot as plt
import pickle
import copy
import time

- ##################
- Demonstrate all types of softmax architectures available
1. a separate folder may be made for every single class, then all of the folders may be placed together within the root folder
2. the folders may be clustered, with each clustered group of folders placed into an outer folder, with the umbrella network predicting only which cluster each image belongs to
3. same as above, but the umbrella network predicts first the cluster, then the specific class within the predicted subcluster
4. same as above, but the folders are clustered into a large number of outer level folders, then the outer level folders are also clustered into the top level folders, with the umbrella network predicting just the outer level cluster
5. (not shown) the last example could be repeated with an umbrella network that predicts every level of the folder structure
- ##################

SOFTMAX ON SMALL MULTI-LAYER STRUCTURE COMPRESSED INTO TOP LEVEL FOLDERS
- the root folder contains 3 inner folders, each of which contains several inner folders
- layers is set to 0, so the program makes a single softmax model from the last level folders

In [5]:
# the small_hierarchy folder has a just 3 outer folders, each with some inner folders
network_type = 'softmax'
images_folder = "/content/drive/My Drive/5390/project/small_hierarchy"
serial_file = "/content/drive/My Drive/Colab Notebooks/small_hierarchy.dat"
models_folder = "/content/drive/My Drive/Colab Notebooks/models"
target_image_size = 150
levels = 0
channels = 3
valid_ratio = 0.2 # validation set
test_ratio = 0.0 # test set
initialize_now = True
SOFTMAX = Umbrella_Network(network_type,images_folder,serial_file,models_folder,target_image_size,channels,levels,valid_ratio,test_ratio,initialize_now)


In [6]:
SOFTMAX.traverse_training_data()

small_hierarchy trainX 137 (137,) trainY 137 (137, 10)
   alligator trainX 0 (0,) trainY 0 (0,)
   mammals trainX 0 (0,) trainY 0 (0,)
   salamanders trainX 0 (0,) trainY 0 (0,)
   snake trainX 0 (0,) trainY 0 (0,)
   automobiles trainX 0 (0,) trainY 0 (0,)
   bicycles trainX 0 (0,) trainY 0 (0,)
   boats trainX 0 (0,) trainY 0 (0,)
   cactus trainX 0 (0,) trainY 0 (0,)
   hedges trainX 0 (0,) trainY 0 (0,)
   shrubs trainX 0 (0,) trainY 0 (0,)


In [7]:
SOFTMAX.make_models()

making model for small_hierarchy


In [8]:
SOFTMAX.train_models(eps=1,batch=10,aug=False,vrb=True)

training model for small_hierarchy
trainX 137
trainY 137
14/14 [==============================] - 10s 18ms/step - loss: 2.9166 - accuracy: 0.1095


In [9]:
SOFTMAX.get_training_accuracy()
SOFTMAX.predict_validation_set() # set verbose=True to view all the images
SOFTMAX.predict_validation_top5()

average training accuracy 0.109489
average training loss 2.916621
34

average validation accuracy 0.058824
top 5 accuracy 0.65


SOFTMAX UMBRELLA ON SMALL MULTI-LAYER STRUCTURE WITHOUT COMPRESSION OF LOWER LAYERS
- the root folder contains 3 inner folders, each of which contains several inner folders, which makes a total of 2 layers
- layers is set to 2, so the program makes 1 outer softmax model and 3 inner softmax models, one for each outer model

In [10]:
# the small_hierarchy folder has a just 3 outer folders, each with some inner folders
network_type = 'softmax'
images_folder = "/content/drive/My Drive/5390/project/small_hierarchy"
serial_file = "/content/drive/My Drive/Colab Notebooks/small_hierarchy.dat"
models_folder = "/content/drive/My Drive/Colab Notebooks/models"
target_image_size = 150
channels = 3
levels = 2
valid_ratio = 0.2 # validation set
test_ratio = 0.0 # test set
initialize_now = True
SOFTMAX = Umbrella_Network(network_type,images_folder,serial_file,models_folder,target_image_size,channels,levels,valid_ratio,test_ratio,initialize_now)


In [11]:
SOFTMAX.traverse_training_data()

small_hierarchy trainX 137 (137,) trainY 137 (137, 3)
   animals trainX 35 (35,) trainY 35 (35, 4)
      alligator trainX 0 (0,) trainY 0 (0,)
      mammals trainX 0 (0,) trainY 0 (0,)
      salamanders trainX 0 (0,) trainY 0 (0,)
      snake trainX 0 (0,) trainY 0 (0,)
   objects trainX 51 (51,) trainY 51 (51, 3)
      automobiles trainX 0 (0,) trainY 0 (0,)
      bicycles trainX 0 (0,) trainY 0 (0,)
      boats trainX 0 (0,) trainY 0 (0,)
   plants trainX 51 (51,) trainY 51 (51, 3)
      cactus trainX 0 (0,) trainY 0 (0,)
      hedges trainX 0 (0,) trainY 0 (0,)
      shrubs trainX 0 (0,) trainY 0 (0,)


In [12]:
SOFTMAX.make_models()

making model for small_hierarchy
making model for animals
making model for objects
making model for plants


In [13]:
SOFTMAX.train_models(eps=1,batch=10,aug=False,vrb=True)

training model for small_hierarchy
trainX 137
trainY 137
14/14 [==============================] - 0s 11ms/step - loss: 1.1588 - accuracy: 0.3650
training model for animals
trainX 35
trainY 35
4/4 [==============================] - 0s 46ms/step - loss: 3.3378 - accuracy: 0.2000
training model for objects
trainX 51
trainY 51
6/6 [==============================] - 1s 35ms/step - loss: 3.3078 - accuracy: 0.2353
training model for plants
trainX 51
trainY 51
6/6 [==============================] - 0s 10ms/step - loss: 1.6077 - accuracy: 0.3529


In [14]:
SOFTMAX.get_training_accuracy()
SOFTMAX.predict_validation_set() # set verbose=True to view all the images
SOFTMAX.predict_validation_top5()

average training accuracy 0.288300
average training loss 2.353020
34

average validation accuracy 0.029412
top 5 accuracy 0.62
